In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# Python version: 3.6


from tqdm import tqdm
import matplotlib.pyplot as plt
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import torch
from torch import nn
from torch.utils.data import DataLoader

from utils import get_dataset
# from options import args_parser
from update import test_inference
from models import MLP, CNNMnist, CNNFashion_Mnist, CNNCifar

# python src/baseline_main.py --model=mlp --dataset=mnist --epochs=10


In [2]:
class Args(object):
    
    # federated parameters (default values are set)
    epochs = 10
    num_users = 10
    frac = 1 # fraction of clients
    local_ep = 5 # num of local epoch
    local_bs = 128 # batch size
    lr = 0.001
    momentum = 0.9
    
    # model arguments
    model = 'cnn'
    kernel_num = 9 # num of each kind of kernel
    kernel_sizes = '3,4,5' # comma-separated kernel size to use for convolution
    num_channels = 1 # num of channels of imgs
    norm = 'batch_norm' # batch_norm, layer_norm, None
    num_filters = 32 # num of filters for conv nets -- 32 for mini-imagenet, 64 for omiglot
    max_pool = 'True' # whether use max pooling rather than strided convolutions
    
    # other arguments
    dataset = "fmnist"
    num_classes = 10 
    gpu = 0
    optimizer = 'sgd'
    iid = 1 # 0 for non-iid
    unequal = 0 # whether to use unequal data splits for non-iid settings (0 for equal splits)
    stopping_rounds = 10 # rounds of early stopping
    verbose = 1
    seed = 1

    # malicious arguments
    n_attackers = [0]
    attack_type = None #'untargeted_mkrum' # untargeted_med, untargeted_mkrum

In [3]:
# bsz : batch size (number of positive pairs)
# d   : latent dim
# x   : Tensor, shape=[bsz, d]
#       latents for one side of positive pairs
# y   : Tensor, shape=[bsz, d]
#       latents for the other side of positive pairs

def align_loss(x, y, alpha=2):
    return (x - y).norm(p=2, dim=1).pow(alpha).mean()

def uniform_loss(x, t=2):
    return torch.pdist(x, p=2).pow(2).mul(-t).exp().mean().log()

In [4]:

if __name__ == '__main__':
    args = Args()
    device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

    # load datasets
    train_dataset, test_dataset, _ = get_dataset(args)

    # BUILD MODEL
    if args.model == 'cnn':
        # Convolutional neural netork
        if args.dataset == 'mnist':
            global_model = CNNMnist(args=args)
        elif args.dataset == 'fmnist':
            global_model = CNNFashion_Mnist(args=args)
        elif args.dataset == 'cifar':
            global_model = CNNCifar(args=args)
    elif args.model == 'mlp':
        # Multi-layer preceptron
        img_size = train_dataset[0][0].shape
        len_in = 1
        for x in img_size:
            len_in *= x
            global_model = MLP(dim_in=len_in, dim_hidden=64,
                               dim_out=args.num_classes)
    else:
        exit('Error: unrecognized model')

    # Set the model to train and send it to device.
    global_model.to(device)
    global_model.train()
    print(global_model)

    # Training
    # Set optimizer and criterion
    if args.optimizer == 'sgd':
        optimizer = torch.optim.SGD(global_model.parameters(), lr=args.lr,
                                    momentum=0.5)
    elif args.optimizer == 'adam':
        optimizer = torch.optim.Adam(global_model.parameters(), lr=args.lr,
                                     weight_decay=1e-4)

    trainloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    criterion = nn.CrossEntropyLoss().to(device)
    epoch_loss = []

    for epoch in tqdm(range(args.epochs)):
        batch_loss = []

        for batch_idx, (images, labels) in enumerate(trainloader):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = global_model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            if batch_idx % 50 == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch+1, batch_idx * len(images), len(trainloader.dataset),
                    100. * batch_idx / len(trainloader), loss.item()))
            batch_loss.append(loss.item())

        loss_avg = sum(batch_loss)/len(batch_loss)
        print('\nTrain loss:', loss_avg)
        epoch_loss.append(loss_avg)

    # Plot loss
    plt.figure()
    plt.plot(range(len(epoch_loss)), epoch_loss)
    plt.xlabel('epochs')
    plt.ylabel('Train loss')
    plt.savefig('../save/nn_{}_{}_{}.png'.format(args.dataset, args.model,
                                                 args.epochs))

    # testing
    test_acc, test_loss = test_inference(args, global_model, test_dataset)
    print('Test on', len(test_dataset), 'samples')
    print("Test Accuracy: {:.2f}%".format(100*test_acc))

CNNFashion_Mnist(
  (conv1): Conv2d(1, 64, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(64, 64, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=25600, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)


  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.299788
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.162196
Train Epoch: 1 [6400/60000 (11%)]	Loss: 1.791932
Train Epoch: 1 [9600/60000 (16%)]	Loss: 1.277201
Train Epoch: 1 [12800/60000 (21%)]	Loss: 1.177069
Train Epoch: 1 [16000/60000 (27%)]	Loss: 0.787301
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.867628
Train Epoch: 1 [22400/60000 (37%)]	Loss: 0.842936
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.853485
Train Epoch: 1 [28800/60000 (48%)]	Loss: 0.734082
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.902456
Train Epoch: 1 [35200/60000 (59%)]	Loss: 0.918625
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.727262
Train Epoch: 1 [41600/60000 (69%)]	Loss: 0.700110
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.623699
Train Epoch: 1 [48000/60000 (80%)]	Loss: 0.736443
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.785856
Train Epoch: 1 [54400/60000 (91%)]	Loss: 0.954228
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.704179


 10%|████████▎                                                                          | 1/10 [00:15<02:17, 15.32s/it]


Train loss: 1.003701526663705
Train Epoch: 2 [0/60000 (0%)]	Loss: 0.609887
Train Epoch: 2 [3200/60000 (5%)]	Loss: 0.671147
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.582256
Train Epoch: 2 [9600/60000 (16%)]	Loss: 0.684173
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.589314
Train Epoch: 2 [16000/60000 (27%)]	Loss: 0.671896
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.653801
Train Epoch: 2 [22400/60000 (37%)]	Loss: 0.599076
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.751626
Train Epoch: 2 [28800/60000 (48%)]	Loss: 0.622268
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.440624
Train Epoch: 2 [35200/60000 (59%)]	Loss: 0.739971
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.582485
Train Epoch: 2 [41600/60000 (69%)]	Loss: 0.744870
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.728940
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.659028
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.569339
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.620684
Train Epoch: 2 [57600/60000 (96%)]	Loss: 0.757026


 20%|████████████████▌                                                                  | 2/10 [00:27<01:50, 13.75s/it]


Train loss: 0.6446382510128306
Train Epoch: 3 [0/60000 (0%)]	Loss: 0.537952
Train Epoch: 3 [3200/60000 (5%)]	Loss: 0.446707


 20%|████████████████▌                                                                  | 2/10 [00:29<01:56, 14.61s/it]


KeyboardInterrupt: 